# Aula 9
# Resampling e validação

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.model_selection import KFold, cross_validate
from sklearn.linear_model import LogisticRegression

## 1) Cross-validation
Para esse exercício, usaremos uma base de dados das candidaturas à Câmara dos Deputados em 2014 que contém, entre outros, variáveis como o sexo, a raça, a escolaridade e o status de reeleição das candidaturas, bem como uma dummy (resultado) que indica se a candidatura foi () ou não () eleita (Machado, Campos, e Recch 2020).

In [2]:
link = 'https://raw.githubusercontent.com/FLS-6497/datasets/main/aula9/camara_2014.csv'
dados = pd.read_csv(link, sep=';', decimal=",")

## a) Básico
Crie uma pipeline para estandardizar variáveis numéricas (ou transformar variáveis categóricas em dummies) com algum modelo de classificação da sua escolha e o valide usando K-fold com k=5 e, depois, com k=10.

In [15]:
# o que acontece quando passamos variáveis numéricas para o OneHotEncoder?
y = dados.resultado
cat_cols = dados.columns[~dados.columns.isin(['resultado', 'receita_proc_ln'])]
dados_cat = dados[cat_cols]
dados_cat = OneHotEncoder().fit_transform(dados_cat)
dados_num = dados[['receita_proc_ln']]
X = hstack([dados_cat, dados_num])

def cross_validate_kfold_logreg(k):
    kf = KFold(n_splits=k)
    logreg = LogisticRegression()
    return cross_validate(logreg, X, y, cv=kf, scoring='f1')

{'fit_time': array([0.04988265, 0.0330739 , 0.04088688, 0.03407478, 0.01881862]), 'score_time': array([0.0060091 , 0.        , 0.        , 0.00299692, 0.        ]), 'test_score': array([0.60550459, 0.61842105, 0.54320988, 0.69736842, 0.46938776])}
{'fit_time': array([0.02438021, 0.03300786, 0.02688622, 0.04741144, 0.03405905,
       0.05072284, 0.05769563, 0.05773497, 0.04764009, 0.03243589]), 'score_time': array([0.        , 0.        , 0.        , 0.00202632, 0.00051212,
       0.00410867, 0.00784063, 0.        , 0.00705457, 0.        ]), 'test_score': array([0.56896552, 0.6185567 , 0.57894737, 0.66666667, 0.56198347,
       0.57142857, 0.73972603, 0.64935065, 0.46808511, 0.51851852])}


In [16]:
cross_validate_kfold_logreg(k=5)

{'fit_time': array([0.03463507, 0.04138041, 0.03198671, 0.0322566 , 0.02407622]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.60550459, 0.61842105, 0.54320988, 0.69736842, 0.46938776])}

In [19]:
cross_validate_kfold_logreg(k=10)

{'fit_time': array([0.05629539, 0.04067302, 0.04954863, 0.04501462, 0.05291629,
        0.05189085, 0.05719256, 0.04808998, 0.05654693, 0.04931712]),
 'score_time': array([0.00799918, 0.00820661, 0.00228453, 0.0079999 , 0.00052094,
        0.        , 0.00800753, 0.        , 0.        , 0.        ]),
 'test_score': array([0.56896552, 0.6185567 , 0.57894737, 0.66666667, 0.56198347,
        0.57142857, 0.73972603, 0.64935065, 0.46808511, 0.51851852])}

## b) LOO
Sorteie apenas algumas observações do banco completo (50, por exemplo) e, em vez de usar K-fold, desta vez use LOO como estratégia de validação (no mlr3, a função chama-se loo; no sklearn, LeaveOneOut).1

In [25]:
# o que acontece quando passamos variáveis numéricas para o OneHotEncoder?
dados_sample = dados.sample(100)
y_sample = dados_sample.resultado
cat_cols = dados_sample.columns[~dados.columns.isin(['resultado', 'receita_proc_ln'])]
dados_sample_cat = dados_sample[cat_cols]
dados_sample_cat = OneHotEncoder().fit_transform(dados_sample_cat)
dados_sample_num = dados_sample[['receita_proc_ln']]
X_sample = hstack([dados_sample_cat, dados_sample_num])

In [34]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
logreg = LogisticRegression()
resultados = cross_validate(logreg, X_sample, y_sample, cv=loo, scoring='accuracy')
resultados = pd.DataFrame(resultados)
resultados.test_score.hist()

## c) Mantendo balanço
Na base de dados, há muito menos candidaturas eleitas do que não-eleitas. Para evitar que amostras de treino e de teste percam esse balanço original, use K-fold estratificado (no mlr3, basta declarar stratum = variavel na task; no sklearn, use StratifiedKFold).

In [46]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
skf.split(X, y)

<generator object _BaseKFold.split at 0x000001E3AA042C70>

## d) Repetindo o processo
Finalmente, use repeated k-fold para minimizar a variação decorrente do sorteio no particionamento das amostras (no mlr3, com repeated_cv; no sklearn, com RepeatedKFold ou com RepeatedStratifiedKFold).

## Workflow de validação
Para este exercício, precisaremos separar a nossa amostra de uma forma mais próxima daquela usada em projetos reais: treino, teste e validação. Para tanto:

## a) Holdout
Faça um holdout inicial da base, separando 90% dela para treino e teste e 10% para validação.

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

## b) Cross-validation
Com os 90% restanted da base, treine e valide um modelo usando alguma estratégia de cross-validation. Ao final, quando encontrar o melhor modelo, treine ele em todos os 90% das observações e o valide na base de validação com 10% de observações.

## Usando mais dados
Neste exercício, vamos voltar à base de dados climático de São Bernardo do Campo e, com o que aprendemos nas últimas aulas, vamos tentar melhorar nosso desempenho na tarefa de predizer temperatura máxima diária. Carregue a base com:

In [ ]:
link = 'https://raw.githubusercontent.com/jacobwright32/Web_Scraper_AI_Core_Project/bb4865ae568e23ab8fadb6ea58cf117df2164ef3/web%20scraping/Cleaned%20Data/Brazil_Sao%20Bernardo%20Do%20Campo_Cleaned.csv'
dados = pd.read_csv(link)

## a) Novo workflow
Monte um workflow para melhorar o desempenho na tarefa de predizer maximum_temprature. Em particular, considere o seguinte:

Pré-processar variáveis contínuas (minmax ou estandardização);
Reduzir dimensionalidade (PCA ou kernelpca);
Considerar combinações não-lineares (criando polinômios ou usando MARS)
Usar ensemble, inclusive com stacking
Usar uma estratégia de validação que deixe mais dados para treino (K-fold com um  ou )
Considerar a estrutura temporal dos dados (é possível criar uma variável lag de maximum_temprature, o transformar o problema em um de série temporal e usar walk-forward validation)